In [36]:
import os
import re
import sys
import glob
import time
import numpy as np
import pandas as pd

In [ ]:
# Function: to keep the order of terms in detail columns same across GWAS, ClinVar and COSMIC.

In [37]:
result_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/formatted_result/reorder_disease"
data_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/formatted_result/"
data_list = [x for x in glob.glob("%s/*.txt" % data_dir) if "QTL" not in os.path.basename(x)] ## modified 
print(data_list)

['/home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/formatted_result/clinvar.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/formatted_result/COSMIC.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/formatted_result/GWAS.txt']


In [48]:
def get_orginal_1st(in_file, out_file):
#     os.system("cut -f 1 %s > %s" % (in_file, out_file))
    df = pd.read_table(in_file, sep="\t")
    orginal_col, result_list = df.iloc[:, 0].tolist(), []
    if os.path.basename(in_file).startswith("GWAS"):
        new_col = "Position|RS_ID|Gene_Name|Disease/Trait|Pubmed_ID|P_value|Database|Type|TagSNP"
        result_list.append(new_col)
        for x in orginal_col:
            line_list = []
            for y in x.split("@"):
                a = y.split("|")
                line_list.append("|".join([a[0], a[1], "NA", a[3], a[2], a[4], a[5], a[6], a[7]]))
            line_result = "@".join(line_list)
            result_list.append(line_result)
    elif os.path.basename(in_file).startswith("COSMIC"):
        new_col = "Position|RS_ID|Gene_Name|Disease/Trait|Mutation_Description|Mutation_zygosity|FATHMM_prediction|Pubmed_ID"
        result_list.append(new_col)
        for x in orginal_col:
            line_list = []
            for y in x.split("@"):
                a = y.split("|")
                line_list.append("|".join([a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7]]))
            line_result = "@".join(line_list)
            result_list.append(line_result)
    else:
        new_col = "Position|RS_ID|Gene_Name|Disease/Trait|Significant|Accession|Study_name|Review_status"
        result_list.append(new_col)
        for x in orginal_col:
            line_list = []
            for y in x.split("@"):
                a = y.split("|")
                line_list.append("|".join([a[0], a[1], "NA", a[3], a[2], a[4], a[5], a[6]]))
            line_result = "@".join(line_list)
            result_list.append(line_result)
    with open(out_file, 'w') as fw:
        fw.writelines(["%s\n" % x.strip() for x in result_list])
#     return out_file
        
def replace_specific_col(file_1, file_2, out_file):
#     file_1 = '"%s"' % file_1
#     print("awk '{t=$0;getline<%s;$1=t}1' %s > %s" % (file_1, file_2, out_file))
#     os.system("awk '{t=$0;getline<%s;$1=t}1' %s > %s" % (file_1, file_2, out_file)) 
    df_new = pd.read_table(file_2, sep="\t")
    new_col, new_list = df_new.columns[0], df_new.iloc[:, 0]
    # ####
    df = pd.read_table(file_1, sep="\t")
    print(len(df), len(new_list))
    print(len(df.columns))
    df.iloc[:, 0] = new_list
    print(len(df.columns))
#     print(len([new_col] + df.columns[1: ]))
#     print(len(df.columns))
    df.rename(columns={df.columns[0]: df_new.columns[0]}, inplace=True)
    df.to_csv(out_file, sep="\t", index=False)

In [49]:
for disease in data_list:
    basen = os.path.basename(disease)
    print(basen)
    tmp_file, result_file = os.path.join(result_dir, "tmp-%s" % basen), os.path.join(result_dir, basen)
    get_orginal_1st(disease, tmp_file)
    replace_specific_col(disease, tmp_file, result_file)

clinvar.txt
2159 2159
16
16
COSMIC.txt
39 39
16
16
GWAS.txt
1237 1237
16
16


In [26]:
### For test
# test_1 = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/formatted_result/test/test1.txt"
# test_2 = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/formatted_result/test/test2.txt"
# test_3 = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/formatted_result/test/test3.txt"
# test_1 = '"%s"' % test_1
# replace_specific_col(test_1, test_2, test_3)

awk '{t=$0;getline<"/home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/formatted_result/test/test1.txt";$1=t}1' /home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/formatted_result/test/test2.txt > /home/galaxy/project/m6AQTL/data/data_for_website/final_data/search_by_QTL/formatted_result/test/test3.txt
